## **Pemilihan ALgoritma untuk Manajemen Kebun**

# **1️⃣ Import Library dan Unduh Dataset**

In [10]:
!pip install gdown --quiet

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics.pairwise import cosine_similarity
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense
import gdown

# Unduh dataset PlansAndFamily.csv
url1 = "https://drive.google.com/uc?id=1nXnJKn-3TCXBW3URPzmRc7WH3cYQQ6Fe"
gdown.download(url1, "PlansAndFamily.csv", quiet=False)

# Unduh dataset Houseplants.csv
url2 = "https://drive.google.com/uc?id=1tE628fdnq32SR8_LrZxJGoTAZ8bTCFvu"
gdown.download(url2, "Houseplants.csv", quiet=False)

Downloading...
From: https://drive.google.com/uc?id=1nXnJKn-3TCXBW3URPzmRc7WH3cYQQ6Fe
To: /content/PlansAndFamily.csv
100%|██████████| 50.0k/50.0k [00:00<00:00, 23.6MB/s]
Downloading...
From: https://drive.google.com/uc?id=1tE628fdnq32SR8_LrZxJGoTAZ8bTCFvu
To: /content/Houseplants.csv
100%|██████████| 72.2k/72.2k [00:00<00:00, 2.96MB/s]


'Houseplants.csv'

## **2️⃣ Membaca dan Mengeksplorasi Dataset**

In [11]:
# Baca dataset
plantsandfamily = pd.read_csv("PlansAndFamily.csv")
houseplants = pd.read_csv("Houseplants.csv")

# Tampilkan info awal
print("📘 Dataset Plants and Family")
display(plantsandfamily.head())

print("\n🌿 Dataset Houseplants")
display(houseplants.head())

📘 Dataset Plants and Family


,Plant Name,Growth,Soil,Sunlight,Watering,Fertilization Type,Family
0,Aloe Vera,slow,sandy,indirect sunlight,Water weekly,Balanced,Asphodelaceae
1,Basil,fast,well-drained,full sunlight,Keep soil evenly moist,Organic,Lamiaceae
2,Snake Plant,slow,well-drained,indirect sunlight,Water when soil is dry,No,Asparagaceae
3,Lavender,moderate,sandy,full sunlight,Let soil dry between watering,No,Lamiaceae
4,Cactus,slow,sandy,full sunlight,Let soil dry between watering,Low-nitrogen,Cactaceae



🌿 Dataset Houseplants


,latin,family,category,climate,ideallight,toleratedlight,watering,insects,use,tempmax_celsius,tempmin_celsius,temp_avg,combined
0,Aeschynanthus lobianus,Gesneriaceae,Hanging,Tropical,Bright light,Direct sunlight,Keep moist between watering. Can be a bit dry ...,"Mealy bug, Aphid, Thrips","Hanging, Flower, Tertiary",32,14,23.0,Tropical Bright light Direct sunlight Keep moi...
1,Adiantum raddianum,Polypodiaceae,Fern,Tropical,Bright light,Diffused,Keep moist between watering. Must not be dry b...,"Mealy bug, Aphid, Snail","Potted plant, Ground cover, Table top",30,12,21.0,Tropical Bright light Diffused Keep moist betw...
2,Aechmea fatsiata,Bromeliaceae,Bromeliad,Tropical humid,Bright light,Diffused,Water when soil is half dry. Change water in t...,NaN,"Flower, Table top, Tertiary",30,12,21.0,Tropical humid Bright light Diffused Water whe...
3,Agave angustilolia Marginata,Amaryllidaceae,Cactus And Succulent,Tropical,6 or more hours of direct sunlight per day.,Direct sunlight.,Water only when the soil is dry. Must be dry b...,"Scale, Mealy bug","Potted plant, Primary, Secondary",35,5,20.0,Tropical 6 or more hours of direct sunlight pe...
4,Aechmea ramosa,Bromeliaceae,Bromeliad,Subtropical,Bright light,Diffused,Water when soil is half dry. Change water in t...,NaN,"Flower, Table top, Primary",30,12,21.0,Subtropical Bright light Diffused Water when s...


## **3️⃣ Preprocessing & Encoding Skala Numerik**

In [12]:
# Pembersihan tanda koma dan titik di kolom teks
plantsandfamily['Sunlight'] = plantsandfamily['Sunlight'].str.replace('[,.]', '', regex=True).str.strip().str.lower()
plantsandfamily['Watering'] = plantsandfamily['Watering'].str.replace('[,.]', '', regex=True).str.strip().str.lower()

houseplants['ideallight'] = houseplants['ideallight'].astype(str).str.replace('[,.]', '', regex=True).str.strip().str.lower()
houseplants['watering'] = houseplants['watering'].astype(str).str.replace('[,.]', '', regex=True).str.strip().str.lower()

# Buat mapping skala numerik untuk kolom Sunlight
sunlight_map = {
    'indirect sunlight': 1,
    'partial sunlight': 2,
    'full sunlight': 3
}

# Mapping watering berdasarkan intensitas dari paling jarang (1) ke paling sering (11/12)
watering_map_paf = {
    'let soil dry between watering': 1,
    'water when soil is dry': 2,
    'water when topsoil is dry': 3,
    'water when soil feels dry': 4,
    'keep soil slightly moist': 5,
    'water when soil is half dry': 6,
    'water weekly': 7,
    'regular watering': 8,
    'keep soil moist': 9,
    'keep soil consistently moist': 10,
    'keep soil evenly moist': 11,
}

watering_map_hpl = {
    'water only when dry must be dry between watering': 1,
    'must be dry between watering water only when dry': 2,
    'water only when the soil is dry must be dry between watering': 3,
    'water only when dry or when soil is half dry': 4,
    'can be dry between watering water when soil is half dry': 5,
    'water when soil is half dry can be dry between watering': 6,
    'keep moist between watering can be a bit dry between watering': 7,
    'keep moist between watering can dry between watering': 8,
    'water when soil is half dry change water in the vase regularly': 9,
    'change water regularly in the vase water when soil is half dry': 10,
    'keep moist between watering water when soil is half dry': 11,
    'keep moist between watering must not be dry between watering': 12,
}

# Mapping ideallight berdasarkan intensitas dari 1-3
ideallight_map = {
    'prefers bright indirect sunlight': 1,
    'bright light': 2,
    '6 or more hours of direct sunlight per day': 3
}

## **4️⃣ Buat DataFrame Pencocokan Numerik**

In [13]:
# Dataframe ketiga: pencocokanplantsandfamily
pencocokanplantsandfamily = plantsandfamily[['Plant Name', 'Sunlight', 'Watering']].copy()
pencocokanplantsandfamily['Sunlight'] = pencocokanplantsandfamily['Sunlight'].map(sunlight_map)
pencocokanplantsandfamily['Watering'] = pencocokanplantsandfamily['Watering'].map(watering_map_paf)
pencocokanplantsandfamily.dropna(inplace=True)

# Dataframe keempat: pencocokanhouseplants
pencocokanhouseplants = houseplants[['latin', 'ideallight', 'watering']].copy()
pencocokanhouseplants['ideallight'] = pencocokanhouseplants['ideallight'].map(ideallight_map)
pencocokanhouseplants['watering'] = pencocokanhouseplants['watering'].map(watering_map_hpl)
pencocokanhouseplants.dropna(inplace=True)

# Tampilkan hasil akhir
print("🔍 Dataframe Pencocokan PlantsAndFamily:")
display(pencocokanplantsandfamily.head())

print("🔍 Dataframe Pencocokan Houseplants:")
display(pencocokanhouseplants.head())

🔍 Dataframe Pencocokan PlantsAndFamily:


,Plant Name,Sunlight,Watering
0,Aloe Vera,1.0,7.0
1,Basil,3.0,11.0
2,Snake Plant,1.0,2.0
3,Lavender,3.0,1.0
4,Cactus,3.0,1.0


🔍 Dataframe Pencocokan Houseplants:


,latin,ideallight,watering
0,Aeschynanthus lobianus,2,7
1,Adiantum raddianum,2,12
2,Aechmea fatsiata,2,9
3,Agave angustilolia Marginata,3,3
4,Aechmea ramosa,2,9


## **5️⃣ Bangun Model Deep Learning TensorFlow**

In [14]:
def build_autoencoder(input_dim):
    input_layer = Input(shape=(input_dim,))
    encoded = Dense(4, activation='relu')(input_layer)
    decoded = Dense(input_dim, activation='linear')(encoded)

    autoencoder = Model(inputs=input_layer, outputs=decoded)
    encoder = Model(inputs=input_layer, outputs=encoded)

    autoencoder.compile(optimizer='adam', loss='mse')
    return autoencoder, encoder

# Siapkan data numerik untuk model
X_paf = pencocokanplantsandfamily[['Sunlight', 'Watering']].values
X_hpl = pencocokanhouseplants[['ideallight', 'watering']].values

# Bangun autoencoder untuk kedua data
auto_paf, encoder_paf = build_autoencoder(X_paf.shape[1])
auto_hpl, encoder_hpl = build_autoencoder(X_hpl.shape[1])

# Latih model
auto_paf.fit(X_paf, X_paf, epochs=100, verbose=0)
auto_hpl.fit(X_hpl, X_hpl, epochs=100, verbose=0)

# Ambil vektor embedding
embed_paf = encoder_paf.predict(X_paf)
embed_hpl = encoder_hpl.predict(X_hpl)

13/13 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
7/7 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 


## **6️⃣ Implementasi Fungsi Rekomendasi**

In [15]:
def RekomendasiPerawatan(nama_tanaman):
    nama = nama_tanaman.strip().lower()

    # SCENARIO 1 - tanaman di plantsandfamily
    match_paf = plantsandfamily[plantsandfamily['Plant Name'].str.lower() == nama]
    if not match_paf.empty:
        sunlight = match_paf['Sunlight'].values[0]
        watering = match_paf['Watering'].values[0]
        pupuk = match_paf['Fertilization Type'].values[0]

        # Ambil skala numerik dari df pencocokan
        entry = pencocokanplantsandfamily[pencocokanplantsandfamily['Plant Name'].str.lower() == nama]
        if entry.empty:
            return "Data numerik tidak ditemukan untuk tanaman ini."

        encoded_input = encoder_paf.predict(entry[['Sunlight', 'Watering']].values)

        # Hitung cosine similarity
        similarities = cosine_similarity(encoded_input, embed_hpl)
        best_match_index = np.argmax(similarities)
        match_latin = pencocokanhouseplants.iloc[best_match_index]['latin']

        # Ambil data serangga dan fungsi
        match_house = houseplants[houseplants['latin'] == match_latin]
        serangga = match_house['insects'].values[0]
        fungsi = match_house['use'].values[0]

        return {
            'Jenis Cahaya': sunlight,
            'Pola Penyiraman': watering,
            'Tipe Pupuk': pupuk,
            'Serangga': serangga,
            'Fungsi Tanaman': fungsi
        }

    # SCENARIO 2 - tanaman di houseplants
    match_hpl = houseplants[houseplants['latin'].str.lower() == nama]
    if not match_hpl.empty:
        cahaya = match_hpl['ideallight'].values[0]
        siram = match_hpl['watering'].values[0]
        serangga = match_hpl['insects'].values[0]
        fungsi = match_hpl['use'].values[0]

        entry = pencocokanhouseplants[pencocokanhouseplants['latin'].str.lower() == nama]
        if entry.empty:
            return "Data numerik tidak ditemukan untuk tanaman ini."

        encoded_input = encoder_hpl.predict(entry[['ideallight', 'watering']].values)

        # Cari tanaman paling mirip dari plantsandfamily
        similarities = cosine_similarity(encoded_input, embed_paf)
        best_match_index = np.argmax(similarities)
        match_plant = pencocokanplantsandfamily.iloc[best_match_index]['Plant Name']

        pupuk = plantsandfamily[plantsandfamily['Plant Name'] == match_plant]['Fertilization Type'].values[0]

        return {
            'Jenis Cahaya': cahaya,
            'Pola Penyiraman': siram,
            'Tipe Pupuk': pupuk,
            'Serangga': serangga,
            'Fungsi Tanaman': fungsi
        }

    # SCENARIO 3 - tidak ditemukan
    return "belum ada saran perawatan untuk tanaman ini"

## **7️⃣ Contoh Penggunaan**

In [16]:
# # Skenario 1: Tanaman dari plantsandfamily
print("Rekomendasi untuk Aloe Vera:")
print(RekomendasiPerawatan("Aloe Vera"))
print("\n")

# Skenario 2: Tanaman dari houseplants
print("Rekomendasi untuk Adiantum raddianum:")
print(RekomendasiPerawatan("Adiantum raddianum"))
print("\n")

# # Skenario 3: Tanaman tidak ada
print("Rekomendasi untuk Tanaman Fiksi:")
print(RekomendasiPerawatan("Tanaman Fiksi"))

Rekomendasi untuk Aloe Vera:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
{'Jenis Cahaya': 'indirect sunlight', 'Pola Penyiraman': 'water weekly', 'Tipe Pupuk': 'Balanced', 'Serangga': 'Scale, Mealy bug', 'Fungsi Tanaman': 'Potted plant, Primary'}


Rekomendasi untuk Adiantum raddianum:
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
{'Jenis Cahaya': 'bright light', 'Pola Penyiraman': 'keep moist between watering must not be dry between watering', 'Tipe Pupuk': 'Organic', 'Serangga': 'Mealy bug, Aphid, Snail', 'Fungsi Tanaman': 'Potted plant, Ground cover, Table top'}


Rekomendasi untuk Tanaman Fiksi:
belum ada saran perawatan untuk tanaman ini


## **8️⃣ Simpan Model Deep Learning ke File .h5 dan dataframe ke .pkl**

In [17]:
# Simpan model dan encoder ke file .h5
encoder_paf.save('encoder_plantsandfamily.h5')
encoder_hpl.save('encoder_houseplants.h5')

In [18]:
import joblib

# Simpan hasil embed dan dataframe yang sudah diproses
joblib.dump({
    'plantsandfamily': plantsandfamily,
    'houseplants': houseplants,
    'pencocokanplantsandfamily': pencocokanplantsandfamily,
    'pencocokanhouseplants': pencocokanhouseplants,
    'embed_paf': embed_paf,
    'embed_hpl': embed_hpl
}, 'plant_data.pkl')

['plant_data.pkl']